<a href="https://colab.research.google.com/github/BinarySstar/Credit-Card-Segment-Classfication/blob/master/Credit_Card_Segment_Classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount
train 및 test 데이터를 drive에 저장하기 위해 mount를 진행

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# EDA
## 데이터 로드 및 전처리
- parquet 형식의 데이터를 로드
- parquet 파일 단위로 이상치 및 결측치를 탐지하여 이를 제거

In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import os
from glob import glob

base_path = 'drive/MyDrive/고급프로그래밍/train'

for subdir in os.listdir(base_path):
    sub_path = os.path.join(base_path, subdir)
    save_path = os.path.join("drive/MyDrive/고급프로그래밍/전처리완료", subdir)
    os.makedirs(save_path, exist_ok=True)

    if os.path.isdir(sub_path):
        parquet_files = glob(os.path.join(sub_path, "*.parquet"))

        for file in parquet_files:
            df = pd.read_parquet(file)

            # 예시: 결측치 제거 + 이상치 제거 (단순 Z-score 방식)
            df_clean = df.dropna()
            for col in df_clean.select_dtypes(include='number').columns:
                mean = df_clean[col].mean()
                std = df_clean[col].std()
                df_clean = df_clean[(df_clean[col] - mean).abs() < 3 * std]

            # 처리된 파일 저장
            fname = os.path.basename(file)
            df_clean.to_parquet(os.path.join(save_path, fname), index=False)

- 전처리된 parquet 파일에 대해 점검 진행

In [2]:
# 전처리된 데이터가 저장된 경로
cleaned_base_path = "drive/MyDrive/고급프로그래밍/전처리완료"

# 이상치 기준 설정 (예: Z-score > 3)
z_threshold = 3

# 이상치/결측치 존재 여부 기록
check_results = {}

for subdir in os.listdir(cleaned_base_path):
    sub_path = os.path.join(cleaned_base_path, subdir)

    if os.path.isdir(sub_path):
        parquet_files = glob(os.path.join(sub_path, "*.parquet"))
        has_missing = False
        has_outliers = False

        for file in parquet_files:
            try:
                df = pd.read_parquet(file)

                # 1. 결측치 확인
                if df.isnull().values.any():
                    has_missing = True

                # 2. 이상치 확인 (수치형 컬럼 대상 Z-score 검사)
                numeric_cols = df.select_dtypes(include='number').columns
                for col in numeric_cols:
                    col_z = ((df[col] - df[col].mean()) / df[col].std()).abs()
                    if (col_z > z_threshold).any():
                        has_outliers = True
                        break  # 하나라도 발견되면 더 안 봐도 됨

                if has_missing or has_outliers:
                    break

            except Exception as e:
                print(f"파일 오류: {file} | {e}")

        check_results[subdir] = {
            '결측치_존재': has_missing,
            '이상치_존재': has_outliers
        }

# 결과 출력
for key, value in check_results.items():
    print(f"[{key}] 결측치: {value['결측치_존재']}, 이상치: {value['이상치_존재']}")

[8.성과정보] 결측치: False, 이상치: False
[7.마케팅정보] 결측치: False, 이상치: False
[6.채널정보] 결측치: False, 이상치: False
[5.잔액정보] 결측치: False, 이상치: False
[4.청구입금정보] 결측치: False, 이상치: False
[3.승인매출정보] 결측치: False, 이상치: False
[2.신용정보] 결측치: False, 이상치: False
[1.회원정보] 결측치: False, 이상치: False
